# EMR on EC2 - Cluster Usage Report 

This notebook provides a detailed view of the costs for a single EMR on EC2 cluster. The dashboard generated at the end of the notebook provides the following information: 

- Hourly costs of the cluster for the time it was running
- AWS Costs repartitioning by service usage (Amazon EC2, Amazon EMR, EBS Storage, Data Transfer)
- Instance Trends to determine which instance types were launched on the cluster (Instance type, family, market, architecture)
- Cost allocation per user and YARN applications, along with unallocated cost view to show resources that were not used

To generate the analytics cost and usage dashboard, please modify the following **Configuration** section to match your environment, and run this notebook.


***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* This notebook uses the `PySpark` kernel.
* The EMR cluster attached to this notebook should have `Spark` `JupyterEnterpriseGateway` `Livy` applications installed.
* The EMR cluster should be configured to use the [AWS Glue Data Catalog as metastore for Spark](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-spark-glue.html).
* To generate a cost breakdown by user or application, the audited cluster should be configured to publish additional YARN utilisation metrics on Amazon S3. See the **Requirements** section in the github repository for the cluster setup. 
* To generate a cost breakdown by user, the EMR Cluster should be configured to use [Kerberos Authentication](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-kerberos.html) or [Hadoop impersonation](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/Superusers.html).
***
## Configuration

In [ ]:
# AWS Glue Database where the AWS Cost and Usage Report is located (e.g. athenacurcfn_a_w_s_costs_daily)
spark.conf.set("cur_db", "YOUR_CUR_DATABASE_NAME")

# AWS Glue Table Name used by AWS Cost and Usage Report (e.g. aws_costs_daily)
spark.conf.set("cur_table", "YOUR_CUR_TABLE_NAME")

# The EMR Cluster you want to audit (e.g. j-2BPMP74G1RN22)
spark.conf.set("cluster.id", "YOUR_EMR_CLUSTER_ID")

# Amazon S3 Bucket name where YARN metrics have been stored. This is the bucket defined in the `emr_usage_reporter.py` script (e.g. mybucket.reports)
spark.conf.set("emr_report_bucket_name", "YOUR_S3_BUCKET_NAME")

# (Optional) Database name used to create the YARN Usage Report Tables.
spark.conf.set("emr_report_db", "emr_usage_report")

# (Optional) Modify this parameter only if you changed the collection interval of the YARN reporter (default: 60 seconds)
spark.conf.set("yarn_metrics_collection_time_sec", "60")

### Create Tables
Tables for YARN metrics can be created using an [AWS Glue Cralwer](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html) specifying the S3 path where the data is stored. As alternative you can create the database and tables required using the following SQL snippets.

In [ ]:
%%sql -q
CREATE DATABASE IF NOT EXISTS ${emr_report_db} LOCATION 's3://${emr_report_bucket_name}/emr_usage_report'

In [ ]:
%%sql -q
CREATE EXTERNAL TABLE IF NOT EXISTS ${emr_report_db}.`application_usage`(
  `id` string COMMENT 'from deserializer', 
  `user` string COMMENT 'from deserializer', 
  `name` string COMMENT 'from deserializer', 
  `queue` string COMMENT 'from deserializer', 
  `state` string COMMENT 'from deserializer', 
  `finalstatus` string COMMENT 'from deserializer', 
  `progress` double COMMENT 'from deserializer', 
  `trackingui` string COMMENT 'from deserializer', 
  `trackingurl` string COMMENT 'from deserializer', 
  `diagnostics` string COMMENT 'from deserializer', 
  `clusterid` bigint COMMENT 'from deserializer', 
  `applicationtype` string COMMENT 'from deserializer', 
  `applicationtags` string COMMENT 'from deserializer', 
  `priority` int COMMENT 'from deserializer', 
  `startedtime` bigint COMMENT 'from deserializer', 
  `launchtime` bigint COMMENT 'from deserializer', 
  `finishedtime` bigint COMMENT 'from deserializer', 
  `elapsedtime` int COMMENT 'from deserializer', 
  `amcontainerlogs` string COMMENT 'from deserializer', 
  `amhosthttpaddress` string COMMENT 'from deserializer', 
  `masternodeid` string COMMENT 'from deserializer', 
  `allocatedmb` int COMMENT 'from deserializer', 
  `allocatedvcores` int COMMENT 'from deserializer', 
  `reservedmb` int COMMENT 'from deserializer', 
  `reservedvcores` int COMMENT 'from deserializer', 
  `runningcontainers` int COMMENT 'from deserializer', 
  `memoryseconds` int COMMENT 'from deserializer', 
  `vcoreseconds` int COMMENT 'from deserializer', 
  `queueusagepercentage` double COMMENT 'from deserializer', 
  `clusterusagepercentage` double COMMENT 'from deserializer', 
  `resourcesecondsmap` struct<entry:struct<key:string,value:string>> COMMENT 'from deserializer', 
  `preemptedresourcemb` int COMMENT 'from deserializer', 
  `preemptedresourcevcores` int COMMENT 'from deserializer', 
  `numnonamcontainerpreempted` int COMMENT 'from deserializer', 
  `numamcontainerpreempted` int COMMENT 'from deserializer', 
  `preemptedmemoryseconds` int COMMENT 'from deserializer', 
  `preemptedvcoreseconds` int COMMENT 'from deserializer', 
  `preemptedresourcesecondsmap` string COMMENT 'from deserializer', 
  `logaggregationstatus` string COMMENT 'from deserializer', 
  `unmanagedapplication` boolean COMMENT 'from deserializer', 
  `amnodelabelexpression` string COMMENT 'from deserializer', 
  `timeouts` struct<timeout:array<struct<type:string,expirytime:string,remainingtimeinseconds:int>>> COMMENT 'from deserializer', 
  `amrpcaddress` string COMMENT 'from deserializer', 
  `resourceinfo` struct<resourceusagesbypartition:array<struct<partitionname:string,used:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>,reserved:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>,pending:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>,amused:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>,amlimit:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>,useramlimit:struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>>>>> COMMENT 'from deserializer')
PARTITIONED BY ( `cluster_id` string)
ROW FORMAT SERDE 
  'org.openx.data.jsonserde.JsonSerDe' 
WITH SERDEPROPERTIES ( 
  'paths'='allocatedMB,allocatedVCores,amContainerLogs,amHostHttpAddress,amNodeLabelExpression,amRPCAddress,applicationTags,applicationType,clusterId,clusterUsagePercentage,diagnostics,elapsedTime,finalStatus,finishedTime,id,launchTime,logAggregationStatus,masterNodeId,memorySeconds,name,numAMContainerPreempted,numNonAMContainerPreempted,preemptedMemorySeconds,preemptedResourceMB,preemptedResourceSecondsMap,preemptedResourceVCores,preemptedVcoreSeconds,priority,progress,queue,queueUsagePercentage,reservedMB,reservedVCores,resourceInfo,resourceSecondsMap,runningContainers,startedTime,state,timeouts,trackingUI,trackingUrl,unmanagedApplication,user,vcoreSeconds') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://${emr_report_bucket_name}/emr_usage_report/application_usage/'
TBLPROPERTIES (
  'classification'='json', 
  'compressionType'='none', 
  'typeOfData'='file'
)

In [ ]:
%%sql -q
CREATE EXTERNAL TABLE IF NOT EXISTS ${emr_report_db}.`cluster_usage`(
  `appssubmitted` int COMMENT 'from deserializer', 
  `appscompleted` int COMMENT 'from deserializer', 
  `appspending` int COMMENT 'from deserializer', 
  `appsrunning` int COMMENT 'from deserializer', 
  `appsfailed` int COMMENT 'from deserializer', 
  `appskilled` int COMMENT 'from deserializer', 
  `reservedmb` int COMMENT 'from deserializer', 
  `availablemb` int COMMENT 'from deserializer', 
  `allocatedmb` int COMMENT 'from deserializer', 
  `pendingmb` int COMMENT 'from deserializer', 
  `reservedvirtualcores` int COMMENT 'from deserializer', 
  `availablevirtualcores` int COMMENT 'from deserializer', 
  `allocatedvirtualcores` int COMMENT 'from deserializer', 
  `pendingvirtualcores` int COMMENT 'from deserializer', 
  `containersallocated` int COMMENT 'from deserializer', 
  `containersreserved` int COMMENT 'from deserializer', 
  `containerspending` int COMMENT 'from deserializer', 
  `totalmb` int COMMENT 'from deserializer', 
  `totalvirtualcores` int COMMENT 'from deserializer', 
  `utilizedmbpercent` int COMMENT 'from deserializer', 
  `utilizedvirtualcorespercent` int COMMENT 'from deserializer', 
  `rmschedulerbusypercent` int COMMENT 'from deserializer', 
  `totalnodes` int COMMENT 'from deserializer', 
  `lostnodes` int COMMENT 'from deserializer', 
  `unhealthynodes` int COMMENT 'from deserializer', 
  `decommissioningnodes` int COMMENT 'from deserializer', 
  `decommissionednodes` int COMMENT 'from deserializer', 
  `rebootednodes` int COMMENT 'from deserializer', 
  `activenodes` int COMMENT 'from deserializer', 
  `shutdownnodes` int COMMENT 'from deserializer', 
  `totalusedresourcesacrosspartition` struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>> COMMENT 'from deserializer', 
  `totalclusterresourcesacrosspartition` struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>> COMMENT 'from deserializer', 
  `totalreservedresourcesacrosspartition` struct<memory:int,vcores:int,resourceinformations:struct<resourceinformation:array<struct<attributes:string,maximumallocation:int,minimumallocation:int,name:string,resourcetype:string,units:string,value:int>>>> COMMENT 'from deserializer', 
  `totalallocatedcontainersacrosspartition` int COMMENT 'from deserializer', 
  `crosspartitionmetricsavailable` boolean COMMENT 'from deserializer', 
  `timestamp` bigint COMMENT 'from deserializer')
PARTITIONED BY ( 
  `cluster_id` string, 
  `year` string, 
  `month` string, 
  `day` string)
ROW FORMAT SERDE 
  'org.openx.data.jsonserde.JsonSerDe' 
WITH SERDEPROPERTIES ( 
  'paths'='activeNodes,allocatedMB,allocatedVirtualCores,appsCompleted,appsFailed,appsKilled,appsPending,appsRunning,appsSubmitted,availableMB,availableVirtualCores,containersAllocated,containersPending,containersReserved,crossPartitionMetricsAvailable,decommissionedNodes,decommissioningNodes,lostNodes,pendingMB,pendingVirtualCores,rebootedNodes,reservedMB,reservedVirtualCores,rmSchedulerBusyPercent,shutdownNodes,timestamp,totalAllocatedContainersAcrossPartition,totalClusterResourcesAcrossPartition,totalMB,totalNodes,totalReservedResourcesAcrossPartition,totalUsedResourcesAcrossPartition,totalVirtualCores,unhealthyNodes,utilizedMBPercent,utilizedVirtualCoresPercent') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://${emr_report_bucket_name}/emr_usage_report/cluster_usage/'
TBLPROPERTIES ( 
  'classification'='json', 
  'compressionType'='none', 
  'typeOfData'='file')

### Refresh partitioned data

Data collected for YARN metrics is partitioned by cluster and date, so it's required to refresh the tables partitioned before querying new data. You can use `MSCK REPAIR` command to automatically add new partitions in the tables.

In [ ]:
%%sql -q
MSCK REPAIR TABLE ${emr_report_db}.`application_usage`

In [ ]:
%%sql -q
MSCK REPAIR TABLE ${emr_report_db}.`cluster_usage`

### Data collection - Cost Report

In [ ]:
%%sql -o cluster_hourly_costs -n -1 -q
SELECT split_part(line_item_resource_id, '/', 2) AS cluster,
       product_product_family                    AS family,
       line_item_usage_start_date - INTERVAL 1 hours AS billing_start,
       sum(line_item_unblended_cost)             AS cost
FROM   ${cur_db}.${cur_table}
WHERE  product_product_name = 'Amazon Elastic MapReduce'
AND    split_part(line_item_resource_id, '/', 2) = '${cluster.id}'
GROUP BY cluster,
         family,
         billing_start 
UNION
SELECT   resource_tags_aws_elasticmapreduce_job_flow_id AS cluster,
         product_product_family                         AS family,
         line_item_usage_start_date                     AS billing_start,
         sum(line_item_unblended_cost)                  AS cost
FROM     ${cur_db}.${cur_table}
WHERE    product_product_name = 'Amazon Elastic Compute Cloud'
AND      resource_tags_aws_elasticmapreduce_job_flow_id NOT IN ('null','applications')
AND      resource_tags_aws_elasticmapreduce_job_flow_id <> ''
AND      resource_tags_aws_elasticmapreduce_job_flow_id = '${cluster.id}'
GROUP BY cluster,
         family,
         billing_start 

In [ ]:
%%sql -o emr_instance_details -n -1 -q
SELECT 
  DISTINCT line_item_resource_id AS instance_id, 
  product_physical_processor AS arch, 
  product_instance_type_family AS instance_family, 
  product_instance_family AS instance_detail, 
  product_instance_type AS instance_type, 
  product_marketoption AS market, 
  1 as num 
FROM 
  ${cur_db}.${cur_table} 
WHERE 
  product_product_name = 'Amazon Elastic Compute Cloud' 
  AND product_product_family = 'Compute Instance' 
  AND resource_tags_aws_elasticmapreduce_job_flow_id NOT IN ('null', 'applications', 'virtualclusters') 
  AND resource_tags_aws_elasticmapreduce_job_flow_id <> ''
  AND resource_tags_aws_elasticmapreduce_job_flow_id = '${cluster.id}'
  AND product_instance_type_family <> ''


### Data collection - Cluster Usage

In [ ]:
%%sql -o yarn_metrics -n -1 -q
SELECT 
  totalmb / 1024 AS totalgb, 
  availablemb / 1024 AS availablegb, 
  allocatedmb / 1024 AS allocatedgb, 
  from_unixtime(
    timestamp / 1000, 'yyyy-MM-dd HH:mm:ss'
  ) AS timestamp 
FROM 
  ${emr_report_db}.cluster_usage 
WHERE 
  cluster_id = '${cluster.id}' 
ORDER BY 
  timestamp

In [ ]:
%%sql -o yarn_application_costs -n -1 -q
SELECT 
  id, 
  user, 
  name, 
  queue, 
  finalstatus, 
  applicationtype, 
  from_unixtime(startedtime / 1000, 'yyyy-MM-dd HH') AS startedtime, 
  from_unixtime(finishedtime / 1000, 'yyyy-MM-dd HH') AS finishedtime, 
  elapsedtime / 1000 AS elapsed_sec, 
  memoryseconds, 
  vcoreseconds, 
  total_memory_mb_avg, 
  memory_sec_cost, 
  memoryseconds * memory_sec_cost AS application_cost 
FROM 
  ${emr_report_db}.application_usage 
  JOIN (
    SELECT 
      billing_start, 
      total_memory_mb_avg, 
      sum(cost) AS total_cost, 
      sum(cost) / (total_memory_mb_avg * minutes_collected * ${yarn_metrics_collection_time_sec}) AS memory_sec_cost 
    FROM 
      (
        SELECT 
          split_part(line_item_resource_id, '/', 2) AS cluster, 
          line_item_usage_start_date - INTERVAL 1 hours AS billing_start, 
          sum(line_item_unblended_cost) AS cost 
        FROM 
          ${cur_db}.${cur_table} 
        WHERE 
          product_product_name = 'Amazon Elastic MapReduce' 
          AND split_part(line_item_resource_id, '/', 2) = '${cluster.id}' 
        GROUP BY 
          cluster, 
          billing_start 
        UNION 
        SELECT 
          resource_tags_aws_elasticmapreduce_job_flow_id AS cluster, 
          line_item_usage_start_date AS billing_start, 
          sum(line_item_unblended_cost) AS cost 
        FROM 
          ${cur_db}.${cur_table} 
        WHERE 
          product_product_name = 'Amazon Elastic Compute Cloud' 
          AND resource_tags_aws_elasticmapreduce_job_flow_id NOT IN ('null', 'applications') 
          AND resource_tags_aws_elasticmapreduce_job_flow_id <> '' 
          AND resource_tags_aws_elasticmapreduce_job_flow_id = '${cluster.id}' 
        GROUP BY 
          cluster, 
          billing_start
      ) AS a 
      JOIN (
        SELECT 
          from_unixtime(timestamp / 1000, 'yyyy-MM-dd HH') AS DAY_HOUR, 
          avg(totalmb) AS total_memory_mb_avg, 
          count(*) AS minutes_collected 
        FROM 
          ${emr_report_db}.cluster_usage 
        WHERE 
          cluster_id = '${cluster.id}' 
        GROUP BY 
          DAY_HOUR
      ) AS b 
    WHERE 
      a.billing_start = b.DAY_HOUR 
    GROUP BY 
      billing_start, 
      total_memory_mb_avg, 
      minutes_collected 
    ORDER BY 
      billing_start
  ) 
WHERE 
  cluster_id = '${cluster.id}' 
  AND from_unixtime(startedtime / 1000, 'yyyy-MM-dd HH') = billing_start

### Dashboards

In [ ]:
%%local
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Default Theme
pio.templates["aws_template"] = go.layout.Template(
    layout_autosize=True,
    layout_height=600
)
pio.templates.default = 'aws_template+gridon'

# Cluster Cost Details
cluster_detailed_costs = cluster_hourly_costs[["cluster", "family","cost"]].groupby(['cluster','family'], as_index=False)['cost'].sum()

## Hourly Cost Dashboard
cluster_hourly_costs_total = cluster_hourly_costs.groupby('billing_start', as_index=False)['cost'].sum()
fig = px.bar(cluster_hourly_costs, x=cluster_hourly_costs.billing_start, y=cluster_hourly_costs.cost.round(2), color=cluster_hourly_costs.family)
fig.add_trace(go.Scatter(x=cluster_hourly_costs_total.billing_start, y=cluster_hourly_costs_total.cost.round(2), text=cluster_hourly_costs_total.cost.round(2), mode='text', textposition='top center', texttemplate = "%{y:$,.2f}",showlegend=False))
fig.update_layout(title='Hourly Cost', xaxis_title="", yaxis_title="Costs ($)")
fig.update_layout(height=500, legend_title_text='', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## Aggregated Cost Dashboard
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "xy"}, {"type": "domain"}]])
fig.add_trace(go.Bar(x=cluster_detailed_costs.family, y=cluster_detailed_costs.cost.round(2), text=cluster_detailed_costs.cost.round(2), texttemplate = "%{y:$,.2f}", showlegend=False, name=""), row=1, col=1)
fig.add_trace(go.Pie(labels=cluster_detailed_costs.family, values=cluster_detailed_costs.cost.round(2), name="", texttemplate = "%{label} <br> %{percent}", textposition = "inside"), row=1, col=2)
fig.update_layout(height=500, showlegend=False, title='Cost Split by Service')
fig.update_yaxes(title_text="Costs ($)", row=1, col=1)
fig.show()

# Cluster Cost Allocation
apps = yarn_application_costs
agg_user_costs = apps.groupby('user', as_index=False)['application_cost'].sum()
agg_apptype_costs = apps.groupby('applicationtype', as_index=False)['application_cost'].sum()
agg_allocated_costs = apps.application_cost.sum()
cluster_total_costs = cluster_detailed_costs.cost.sum()
cluster_idle_costs = cluster_total_costs - agg_allocated_costs

## Instance Trends
instance_market = emr_instance_details[['market', 'num']].groupby(['market'], as_index=False)['num'].sum()
instance_family = emr_instance_details[['instance_family', 'instance_type', 'num']].groupby(['instance_family', 'instance_type'], as_index=False)['num'].sum()
instance_arch = emr_instance_details[['arch', 'num']].groupby(['arch'], as_index=False)['num'].sum()
instance_desc = emr_instance_details[['instance_detail', 'num']].groupby(['instance_detail'], as_index=False)['num'].sum()

fig = make_subplots(rows=1, cols=4, specs=[[{"type": "xy"},{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]])
fig.add_trace(go.Bar(x=instance_family.instance_type, y=instance_family.num, showlegend=False, name="", text=instance_family.num, textposition = "inside"), row=1, col=1)
fig.add_trace(go.Pie(labels=instance_arch.arch, values=instance_arch.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=2)
fig.add_trace(go.Pie(labels=instance_market.market, values=instance_market.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=3)
fig.add_trace(go.Pie(labels=instance_desc.instance_detail, values=instance_desc.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=4)
fig.update_layout(title='EC2 Instance Trends', height=500, showlegend=False)
fig.show()

## YARN Memory Dashboard
fig = go.Figure()
fig.add_scatter(x=yarn_metrics.timestamp, y=yarn_metrics.totalgb, name = 'Total Memory')
fig.add_scatter(x=yarn_metrics.timestamp, y=yarn_metrics.allocatedgb, name = 'Allocated Memory')
fig.update_layout(title="YARN Memory Usage", xaxis_title="", yaxis_title="Memory (GB)")
fig.update_layout(legend_title_text='', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## User Costs Dashboard
fig = make_subplots(rows=1, cols=3, specs=[[{"type": "xy"}, {"type": "domain"},{"type": "table"}]])
fig.add_trace(go.Bar(x=agg_user_costs.user, y=agg_user_costs.application_cost.round(3), showlegend=False, name="", text=agg_user_costs.application_cost.round(3), texttemplate = "%{y:$,.2f}"), row=1, col=1)
fig.add_trace(go.Pie(labels=['Allocated', 'Unallocated'], values=[agg_allocated_costs.round(2), cluster_idle_costs.round(2)], name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside", showlegend=False), row=1, col=2)
fig.add_trace(go.Table(header=dict(values=['Total Cluster Costs ($)','Allocated Costs ($)', 'Unallocated Costs ($)'], align="left"), cells=dict(values=[cluster_total_costs.round(2), agg_allocated_costs.round(2), cluster_idle_costs.round(2)], align = "left")), row=1, col=3)
fig.update_layout(title = 'Cost Allocation', height=500)
fig.update_yaxes(title_text="Costs ($)", row=1, col=1)
fig.show()

## Application Costs Dashboard
fig = go.Figure(data=[go.Table(
    header=dict(values=['Application ID', 'Name', 'User', 'Status', 'Type', 'Memory-Seconds Used', 'vCores-Seconds Used', 'Elapsed Time (seconds)' ,'Costs ($)'], align='left'),
    cells=dict(values=[apps.id, apps.name, apps.user, apps.finalstatus, apps.applicationtype, apps.memoryseconds, apps.vcoreseconds, apps.elapsed_sec.round(2), apps.application_cost.round(2)], align='left'))
])
fig.update_layout(title=f'Application Costs Details',height=800)
fig.show()